In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setting up paths
train_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Test'
test_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Train'
val_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Validation'

In [ ]:
# Function to add Gaussian noise
def add_noise(img):
    noise_factor = 0.1  # Adjust for noise level
    noisy_img = img + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=img.shape)
    noisy_img = np.clip(noisy_img, 0.0, 1.0)  # Ensure values are between 0 and 1
    return noisy_img

In [ ]:
# Custom Data Generator with noise addition
class NoisyImageDataGenerator(Sequence):
    def __init__(self, generator):
        self.generator = generator

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        images, labels = self.generator[index]
        noisy_images = np.array([add_noise(img) for img in images])  # Apply noise
        return noisy_images, labels

In [ ]:
# Data generators
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)
train_generator = NoisyImageDataGenerator(train_generator)  # Add noise

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

In [ ]:
# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the Model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)


In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

In [ ]:
# Confusion Matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

# Example Confusion Matrix Data
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Create a custom color map for each cell
cell_colors = np.array([
    ['#FFCCCC', '#FF6666'],  # Light and dark red for row 1
    ['#66B2FF', '#0033CC']   # Light and dark blue for row 2
])

# Plot the confusion matrix with custom cell colors
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cbar=False, linewidths=1,
            linecolor='black', xticklabels=class_labels, yticklabels=class_labels,
            square=True, cmap=None, annot_kws={"size": 15}, mask=None)

# Loop through each cell to manually set the background color
ax = plt.gca()
for i in range(conf_matrix.shape[0]):  # Rows
    for j in range(conf_matrix.shape[1]):  # Columns
        ax.add_patch(plt.Rectangle((j, i), 1, 1, color=cell_colors[i][j], alpha=0.5))

# Add titles and labels
plt.title('Confusion Matrix ')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(ticks=np.arange(len(class_labels)) + 0.5, labels=class_labels, rotation=45)
plt.yticks(ticks=np.arange(len(class_labels)) + 0.5, labels=class_labels, rotation=45)
plt.tight_layout()
plt.show()




In [ ]:
# Classification Report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Classification Report:")
print(report)

In [ ]:
# Plot Performance
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Check Test Data Predictions (Real or Fake)
test_filenames = test_generator.filenames

# Define the number of images to display
num_images_to_display = 20

# Create a grid layout for better visualization
rows, cols = 4, 5  # Adjust based on the number of images
plt.figure(figsize=(15, 12))

for i in range(num_images_to_display):
    img_path = test_dir + '/' + test_filenames[i]
    img = plt.imread(img_path)

    # Get the prediction
    prediction = 'Real' if predicted_classes[i][0] == 0 else 'Fake'

    # Plot the image
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.title(f"Predicted: {prediction}", fontsize=10)
    plt.axis('off')

# Add a title to the overall grid
plt.suptitle("Test Data Predictions (Real or Fake)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit title
plt.show()


In [ ]:
############# fake ELA

from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt

def perform_ela(image_path, quality=95):



    original = Image.open(image_path).convert('RGB')


    temp_path = 'temp_compressed.jpg'
    original.save(temp_path, 'JPEG', quality=quality)


    compressed = Image.open(temp_path)

    ela_image = ImageChops.difference(original, compressed)


    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0 / max_diff if max_diff != 0 else 1
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    return ela_image

def display_ela(image_path, ela_result):

    original = Image.open(image_path).convert('RGB')

    # Plot original and ELA result
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(np.array(original))
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(np.array(ela_result))
    axes[1].set_title('ELA Result')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage with your specific image path
image_path = '/content/drive/MyDrive/Artificial Intelligence part_1/Test/Fake/fake_1008.jpg'
ela_image = perform_ela(image_path)
display_ela(image_path, ela_image)




In [ ]:
########### REAL
from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt

def perform_ela(image_path, quality=95):



    original = Image.open(image_path).convert('RGB')


    temp_path = 'temp_compressed.jpg'
    original.save(temp_path, 'JPEG', quality=quality)


    compressed = Image.open(temp_path)

    ela_image = ImageChops.difference(original, compressed)


    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0 / max_diff if max_diff != 0 else 1
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    return ela_image

def display_ela(image_path, ela_result):

    original = Image.open(image_path).convert('RGB')

    # Plot original and ELA result
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(np.array(original))
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(np.array(ela_result))
    axes[1].set_title('ELA Result')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage with your specific image path
image_path = '/content/drive/MyDrive/Artificial Intelligence part_1/Test/Real/real_10 - Copy.jpg'
ela_image = perform_ela(image_path)
display_ela(image_path, ela_image)